In [33]:
from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy import dataset_collections as collections
import pandas as pd
from io import StringIO
import os
import requests

In [19]:
%env GALAXY_API_KEY=cf4092f19ab187b157fea51f97f2791b

env: GALAXY_API_KEY=cf4092f19ab187b157fea51f97f2791b


In [20]:
gi = GalaxyInstance(url='https://usegalaxy.eu/', key=os.environ['GALAXY_API_KEY'])

### Get history or create one

In [38]:
h_name = 'collection_upload'
histories = gi.histories.get_histories(name = h_name)
if not histories:
    history_params = gi.histories.create_history(h_name)
    history_id = history_params["id"]
elif len(histories) > 1:
    raise ValueError('Too many histories with that name')
else:
    history_id = histories[0]["id"]

print(history_id)

0cea14756c402113


### Upload files

In [39]:
files = os.path.join('collection_param_input/')
dataset_ids = {}
for file in os.listdir(files):
    if '.txt' in file:
        file_path = os.path.join(files, file)
        dataset = gi.tools.upload_file(file_path, history_id)
        dataset_ids[dataset['outputs'][0]['id']] = dataset['outputs'][0]['name']

### Create collection

In [40]:
elements = []
for hid, name in dataset_ids.items():
    elements.append(collections.HistoryDatasetElement(name=name, id=hid))

collection_response = gi.histories.create_dataset_collection(
            history_id=history_id,
            collection_description=collections.CollectionDescription(
                name="MyTestPair",
                type="list",
                elements=elements
            )
    )